# **Data Wrangling**

> The purpose of this script is to modify the bus data obtained from the STM and the historical weather data obtained from the Government of Canada. This script is not necessary to be run as the modified data is available upon download/clone of the repository.

In [1]:
import os
import pandas as pd
import numpy as np
import time
import urllib.request
import glob
from datetime import datetime, timedelta

## STM Data

> This data was provided by the STM under the terms of the *Act respecting access to documents held by public bodies and the protection of personal information (R.S.Q., chapter A-2.1)*.

In [ ]:
# Merge STM files and save as a pandas dataframe

STM_1_df = pd.read_csv('../Data/Transit data/STM_Data_2021_2022.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
STM_1_df = STM_1_df.dropna(how='all')
STM_2_df = pd.read_csv('../Data/Transit data/STM_Data_2023.csv', dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})

STM_df = pd.concat([STM_1_df, STM_2_df], ignore_index=True)

In [ ]:
STM_df.head()

In [ ]:
# Remove duplicate rows in case the index was copied over

original_df = STM_df.copy()
STM_df = STM_df.drop_duplicates()

removed_rows = original_df[~original_df.index.isin(STM_df.index)]
print(removed_rows)

In [ ]:
# Change data format from YYYY/MM/DD to YYYY-MM-DD
try:
    STM_df['date'] = STM_df['date'].str.replace('/', '-')
    STM_df['date'] = pd.to_datetime(STM_df['date'],errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Date: {e}")

# Verify that the date format is correct
print(STM_df['date'].head())

# Change time format from HH:MM:SS to YYYY-MM-DD HH:MM:SS asigning same date from the date column

try:
    for col in ['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']:
        STM_df[col] = pd.to_datetime(STM_df[col], format='%H:%M:%S').dt.time
        STM_df[col] = STM_df[col].astype(str)
        STM_df[col] = STM_df['date'].astype(str) + ' ' + STM_df[col]
        STM_df[col] = pd.to_datetime(STM_df[col], errors='coerce')
except Exception as e:
    print(f"An error occurred while converting the Time: {e}")

#if the time in pl column is after 23:00 and in rl column is before 1:00, add 1 day to the date in rl column
#if the time in pl column is before 1:00 and in rl column is after 23:00, substract 1 day to the date in rl column
#this is to account for arrivals that were scheduled for late night but actually occurred early morning
#and for arrivals that were scheduled for early morning but actually occurred late the previous night

def adjust_dates(row):
    if row['dep_pl'].hour >= 23 and row['dep_rl'].hour < 1:
        row['dep_rl'] += pd.Timedelta(days=1)
    elif row['dep_pl'].hour < 1 and row['dep_rl'].hour >= 23:
        row['dep_rl'] -= pd.Timedelta(days=1)

    if row['arr_pl'].hour >= 23 and row['arr_rl'].hour < 1:
        row['arr_rl'] += pd.Timedelta(days=1)
    elif row['arr_pl'].hour < 1 and row['arr_rl'].hour >= 23:
        row['arr_rl'] -= pd.Timedelta(days=1)
    
    return row

STM_df = STM_df.apply(adjust_dates, axis=1)

<div style="background-color:rgba(255, 165, 0, 0.10); padding:10px;">
Enable this cell to store the STM data as a csv file
</div>

<div style="background-color:rgba(255, 165, 0, 0.10); padding:10px;">
</div>

## Weather Data Manipulation

1) Daily data for snow

<div style="background-color:rgba(255, 165, 0, 0.10); padding:10px;">
This code was used to retrieve all weather data from the Government of Canada website
</div>

2) Hourly data for precipitation

## Master data

In [67]:
from datetime import datetime, timedelta

In [68]:
# Create a new dataframe for the master data with columns from the stm data
STM_df = pd.read_csv('../Data/Transit data/STM_Data.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str,8:str})
df_master = STM_df[['id','date', 'ligne', 'dep_rl', 'arr_rl']]
df_hourlyw = pd.read_csv('../Data/Weather Data/hourly_montreal_weather.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})
df_dailyw = pd.read_csv('../Data/Weather Data/daily_montreal_weather.csv',  dtype={0: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str})

df_master = df_master.rename(columns={'dep_rl': 'start_time', 'arr_rl': 'end_time', 'ligne': 'line'})
df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'], errors='coerce')
df_dailyw['date'] = pd.to_datetime(df_dailyw['date'], errors='coerce')

#cut the date from the start_time column and keep only the time as HH:MM
for col in ['start_time', 'end_time']:
    df_master[col] = df_master[col].astype(str)
    df_master[col] = df_master[col].str.slice(start=11, stop=16)
    df_master[col] = pd.to_datetime(df_master[col], format='%H:%M', errors='coerce').dt.time

df_master.head()

,id,date,line,start_time,end_time
0,.1,2021-10-05,100.0,00:10:00,00:44:00
1,.2,2021-10-05,100.0,00:42:00,01:14:00
2,.3,2021-10-05,100.0,01:13:00,01:44:00
3,.4,2021-10-05,100.0,05:06:00,05:39:00
4,.5,2021-10-05,100.0,05:41:00,06:13:00


In [69]:
#find nan values in the start_time column
df_master[df_master['start_time'].isnull()]
df_master = df_master.dropna(subset=['start_time'])


In [70]:
# Add a actual temperature at start column (temp_start) to the master data, using the hourly weather data which matches the date and hour of the start time
# Must create an hour columm in the master data to match the time column in the weather data (HH:MM)
# Must match the start hour with the next hour in the weather data since the weather time is the end of the hour

df_master['hour'] = df_master['start_time'].astype(str)
df_master['hour'] = df_master['hour'].str.slice(start=0, stop=2)
df_master['hour'] = df_master['hour'].astype(int)
df_master['hour'] = df_master['hour'] + 1
df_master['hour'] = df_master['hour'].astype(str)
df_master['hour'] = df_master['hour'].str.pad(width=2, side='left', fillchar='0')
df_master['hour'] = df_master['hour'] + ':00'

#rename the time column in hourly data to hour
df_hourlyw = df_hourlyw.rename(columns={'time': 'hour'})



In [71]:
df_master.head()

,id,date,line,start_time,end_time,hour
0,.1,2021-10-05,100.0,00:10:00,00:44:00,01:00
1,.2,2021-10-05,100.0,00:42:00,01:14:00,01:00
2,.3,2021-10-05,100.0,01:13:00,01:44:00,02:00
3,.4,2021-10-05,100.0,05:06:00,05:39:00,06:00
4,.5,2021-10-05,100.0,05:41:00,06:13:00,06:00


In [72]:
df_hourlyw.head()

,date,hour,temp,precip
0,2021-10-05,00:00,15.2,0.0
1,2021-10-05,01:00,15.1,0.0
2,2021-10-05,02:00,13.7,0.0
3,2021-10-05,03:00,13.2,0.0
4,2021-10-05,04:00,12.4,0.0


In [73]:
df_master['date'] = pd.to_datetime(df_master['date'])
df_hourlyw['date'] = pd.to_datetime(df_hourlyw['date'])

df_master = pd.merge(df_master, df_hourlyw, how='left', on=['date', 'hour'])
df_master = df_master.rename(columns={'temp': 'temp_start'})
df_master = df_master.drop(columns=['hour'])

df_master.head()

,id,date,line,start_time,end_time,temp_start,precip
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0


In [74]:
# Add rain and snow amount columns to the master data, using the daily weather data which matches the date
# For the snow_amt column, if the snow_yn column is Y, then take the value of the precip column, otherwise set to 0
# For the rain_amt column, if the snow_yn column is N, then take the value of the precip column, otherwise set to 0

df_master = pd.merge(df_master, df_dailyw, how='left', left_on=['date'], right_on=['date'])
df_master = df_master.rename(columns={'precip': 'rain_amt', 'snow': 'total_snow'})
df_master['snow_amt'] = np.where(df_master['snow_yn'] == 'Y', df_master['rain_amt'], 0)
df_master['rain_amt'] = np.where(df_master['snow_yn'] == 'N', df_master['rain_amt'], 0)
df_master = df_master.drop(columns=['snow_yn',])

df_master.head()


,id,date,line,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0,0.0,0
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0,0.0,0
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0,0.0,0
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0,0.0,0
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0,0.0,0


In [75]:
#import dep_pl, dep_rl, arr_pl, arr_rl columns from the stm data to the master data
# cut the seconds off from the time columns

df_master = pd.merge(df_master, STM_df[['id', 'dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']], how='left', left_on=['id'], right_on=['id'])

for col in ['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl']:
    df_master[col] = df_master[col].astype(str)
    df_master[col] = df_master[col].str.slice(start=0, stop=16)
    df_master[col] = df_master[col] + ':00'
    df_master[col] = pd.to_datetime(df_master[col], format='%Y-%m-%d %H:%M:%S', errors='coerce')

#find the delays in minutes between the planned and real departure times in minutes
df_master['delay_start'] = (df_master['dep_rl'] - df_master['dep_pl']).dt.total_seconds() / 60
df_master['delay_start'] = df_master['delay_start'].astype(int)

df_master['delay_end'] = (df_master['arr_rl'] - df_master['arr_pl']).dt.total_seconds() / 60
df_master['delay_end'] = df_master['delay_end'].astype(int)

df_master = df_master.drop(columns=['dep_pl', 'dep_rl', 'arr_pl', 'arr_rl'])

df_master.head()


,id,date,line,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt,delay_start,delay_end
0,.1,2021-10-05,100.0,00:10:00,00:44:00,15.1,0.0,0.0,0,0,-1
1,.2,2021-10-05,100.0,00:42:00,01:14:00,15.1,0.0,0.0,0,0,-3
2,.3,2021-10-05,100.0,01:13:00,01:44:00,13.7,0.0,0.0,0,-1,-5
3,.4,2021-10-05,100.0,05:06:00,05:39:00,11.8,0.0,0.0,0,0,-2
4,.5,2021-10-05,100.0,05:41:00,06:13:00,11.8,0.0,0.0,0,2,-1


In [76]:
#reorganize columns

cols = df_master.columns.tolist()
id_col = cols[0]
date_col = cols[1]
line_col = cols[2]
delay_start_col = cols[9]
cols = [col for col in cols if col not in {id_col,date_col, line_col, delay_start_col}]

cols = [id_col,date_col, line_col, delay_start_col] + cols
df_master = df_master[cols]

df_master.head()


,id,date,line,delay_start,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt,delay_end
0,.1,2021-10-05,100.0,0,00:10:00,00:44:00,15.1,0.0,0.0,0,-1
1,.2,2021-10-05,100.0,0,00:42:00,01:14:00,15.1,0.0,0.0,0,-3
2,.3,2021-10-05,100.0,-1,01:13:00,01:44:00,13.7,0.0,0.0,0,-5
3,.4,2021-10-05,100.0,0,05:06:00,05:39:00,11.8,0.0,0.0,0,-2
4,.5,2021-10-05,100.0,2,05:41:00,06:13:00,11.8,0.0,0.0,0,-1


In [77]:
# verify that there are no delays whose absolute value is greater than 10h
# print the whole list and whole row instead of truncating
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(df_master.loc[abs(df_master['delay_start']) > 600])
print(df_master.loc[abs(df_master['delay_end']) > 600])

             id       date   line  delay_start start_time  end_time  \
14637    .14638 2021-10-20  139.0         -834   07:50:00  22:32:00   
39737    .39719 2021-11-15  460.0         -706   07:45:00  20:29:00   
167836  .167821 2022-03-20  139.0         1018   17:11:00  00:51:00   
211850  .211835 2022-05-03   67.0         1326   22:35:00  01:06:00   
271315  .271300 2022-07-06  460.0         -629   06:16:00  18:12:00   
314365  .314350 2022-08-24   80.0         1328   23:08:00  01:32:00   
361995  .361980 2022-10-15  121.0         1301   21:42:00  00:37:00   
538933  .538897 2023-04-08   80.0         1352   23:50:00  01:43:00   

       temp_start rain_amt  total_snow snow_amt  delay_end  
14637         6.9      0.0         0.0        0          2  
39737         3.0      0.2         0.0        0         -2  
167836        4.2        0         2.0      0.8         -1  
211850       14.2      0.0         0.0        0          0  
271315       16.8      0.0         0.0        0        

In [78]:
# show unique line values
print(df_master['line'].unique())
print(STM_df['ligne'].unique())
#seems like the STM did not give me data for the 193 line ._.

['100.0' '121.0' '139.0' '439.0' '460.0' '467.0' '480.0' '67.0' '80.0']
['100.0' '121.0' '139.0' '439.0' '460.0' '467.0' '480.0' '67.0' '80.0']


In [79]:
df_master.head()

,id,date,line,delay_start,start_time,end_time,temp_start,rain_amt,total_snow,snow_amt,delay_end
0,.1,2021-10-05,100.0,0,00:10:00,00:44:00,15.1,0.0,0.0,0,-1
1,.2,2021-10-05,100.0,0,00:42:00,01:14:00,15.1,0.0,0.0,0,-3
2,.3,2021-10-05,100.0,-1,01:13:00,01:44:00,13.7,0.0,0.0,0,-5
3,.4,2021-10-05,100.0,0,05:06:00,05:39:00,11.8,0.0,0.0,0,-2
4,.5,2021-10-05,100.0,2,05:41:00,06:13:00,11.8,0.0,0.0,0,-1


In [80]:
df_master.to_csv('../Data/master_data.csv', index=False)